In [23]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments,PreTrainedModel,PreTrainedTokenizer
from typing import Tuple
from datasets import Dataset
import random
import torch
from torch.utils.data import DataLoader

batch_size = 32
model_name = 'gpt2'
model = GPT2LMHeadModel.from_pretrained(model_name)

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model.to(device)

tokenizer = GPT2Tokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token  # Example: Using eos_token for padding

In [2]:
!pip install accelerate -U


[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: pip install --upgrade pip


In [21]:
## Load text
def load_text(path:str) -> str:
    with open(path, 'r') as file:
        # Read the entire content of the file into a string
        file_content = file.read()
    return file_content


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


def generate_datasets(text:str)-> Tuple[Dataset,Dataset]:
    '''
    Function to generate tokenized datasets for hugging face models.
    
    :param text: A single text corpus that should be used for the task.
    :type text: str
    :return: Two datasets, where the first is the train_dataset and the second is the test_dataset
    :rtype: Tuple[Dataset,Dataset]
    '''
    
    sentences = text.split('.')
    random.shuffle(sentences)
    split = int(0.9 * len(sentences))
    train_sentences = sentences[:split]
    val_sentences = sentences[split:]

    train_dataset = Dataset.from_dict({"text": train_sentences})
    val_dataset = Dataset.from_dict({"text": val_sentences})
    
    train_dataset = train_dataset.map(tokenize_function, batched=True, remove_columns=["text"])
    val_dataset = val_dataset.map(tokenize_function, batched=True, remove_columns=["text"])
    
    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)



    return train_dataset, val_dataset
    

In [4]:
def fine_tune_model(model:PreTrainedModel,train_dataset:Dataset,val_dataset:Dataset) -> Tuple[PreTrainedModel,PreTrainedTokenizer]:
    '''
    Function to fine tune the pre-trained model on the given dataset

    :param path: Path to the corpus with which the model should be fine-tuned  
    :type path: String 
    :return: Tupel of pre-trained model and tokenizer
    :rtype: Tupel[Model,Tokenizer]
    '''    
    
    training_args = TrainingArguments(
        output_dir="./results",
        evaluation_strategy="epoch",
        learning_rate=2e-5,
        per_device_train_batch_size=2,
        per_device_eval_batch_size=2,
        num_train_epochs=3,
        weight_decay=0.01,
    ) 
    
    ## Load Trainig data:         
    trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset, 
    )
    
    trainer.train()
    
    model.save_pretrained("./fine-tuned-gpt2")
    tokenizer.save_pretrained("./fine-tuned-gpt2")  
    
    return model, tokenizer


In [14]:
import torch

def predict_next_word(text, model, tokenizer, device, num_predictions=1):
    # Tokenize input text
    inputs = tokenizer.encode(text, return_tensors='pt').to(device)
    
    # Model inference
    with torch.no_grad():
        outputs = model(inputs)
    
    # Get logits and probabilities
    logits = outputs.logits
    last_token_logits = logits[:, -1, :]
    probabilities = torch.softmax(last_token_logits, dim=-1)
    
    # Get top predicted indices
    predicted_indices = torch.topk(probabilities, num_predictions).indices.squeeze(0).tolist()
    
    # Convert indices to tokens
    predicted_tokens = [tokenizer.decode([index]) for index in predicted_indices]
    
    return predicted_tokens



In [22]:
train_dataset,test_dataset = generate_datasets(load_text('../data/The_critique_of_pure_reason_short.txt'))

text = 'But it was found that—although it was affirmed that this\nso-called queen could not refer her descent to any higher source than\nthat of common experience, a circumstance which necessarily brought\nsuspicion on her claims—as this _genealogy_ was incorrect, she\npersisted in the advancement of her claims to'
model_test,tokenizer_test = fine_tune_model(model,train_dataset,test_dataset)
pred = predict_next_word(text,model_test,tokenizer_test,device,3)
print(pred)
 


Map:   0%|          | 0/11 [00:00<?, ? examples/s]

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

AttributeError: 'Dataset' object has no attribute 'items'